# Installation und Import

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import plotly.express as px
import numpy as np

# Zusatzmaterial

In diesem Abgschnitt gibt es einige Python-Methoden, die in der Haupterklärung nicht ausführlich behandelt wurden. Diese Methoden können jedoch nützlich sein, wenn Du tiefer in die Funktionsweise des Codes eintauchen möchtest.

In [ ]:
def show_example(X, y, index):
    """
    Zeigt ein Bild aus einer Sammlung von Bildern zusammen mit seiner Beschriftung und Pixelstatistik an.
    
    Args:
    - X: Ein numpy-Array der Form (n, Höhe, Breite), das die Bilder darstellt.
    - y: Ein numpy array of shape (n, ), das die entsprechenden Labels darstellt.
    - index: Eine ganze Zahl, die den Index des anzuzeigenden Bildes angibt.
    
    Rückgabe:
    - Keine
    
    Nebeneffekte:
    - Druckt die Form des Bildes.
    - Druckt die Bezeichnung des Bildes.
    - Gibt den Wert des mittleren Pixels des Bildes aus.
    - Druckt die minimalen und maximalen Pixelwerte des Bildes.
    - Zeigt das Bild mit Hilfe der Plotly Express-Bibliothek an.
    """
    
    image = X[index]
    label = y[index]
    print("Struktur vom Bild:", image.shape)
    print("Label:", y[index])
    print("Pixel in der Mitte:", image[int(image.shape[0]/2), int(image.shape[1]/2)])
    print("Farbwerte (dunkel -> hell):", image.min(), "to", image.max())
    fig = px.imshow(image, binary_string=True)
    if 'google.colab' in str(get_ipython()):
        fig.show(renderer='colab') 
    else:
        fig.show(renderer='iframe') 

    return np.array([image])

# Der MNIST Datensatz

Der [MNIST](!http://yann.lecun.com/exdb/mnist/) Datensatz ist ein wahrer Klassiker und ein absolutes Muss für jeden, der sich für maschinelles Lernen interessiert!

Dieser besteht aus einer großen Sammlung handgeschriebener Ziffern.
Besonders an den Ziffern ist, dass diese auf winzigen 28x28 Pixel Bildern in Graustufen dargestellt sind.

Insgesamt kannst Du dort alle Zahlen von 0 bis 9 in einer Sammlung von 70.000 dieser Bilder finden.

Schön daran ist, dass 60.000 der Ziffern speziell zum Trainieren künstlicher neuronaler Netze verwendet werden können, während die restlichen 10.000 für Tests bereitstehen.

Weitere Informationen kannst Du aus der Dokumentation entnehmen.
Rufe Deine Befehle dazu einfach mit `??` am Ende auf, wie beispielsweise:



```
datasets.mnist.load_data??
```



---
> **_Achtung:_**  Wir beschränken uns in erster Linie auf den Aufbau eines neuronalen Netzes, daher sind die Trainings- und Test-Daten in erster Linie rein repräsentativ.

Also lass uns einen Blick auf die Daten werfen:


In [ ]:
# Zuerst werden die Daten geladen
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [ ]:
# Mit .shape kannst Du sehen wie der Aufbau der Daten ist.
# Du kannst sehen, dass der erste Eintrag die Gesamtzahl der Daten angibt.
# Die darauf folgenden zwei Werte hingegen geben die Größe (Höhe, Breite) der Bilder an.
print("Trainings Daten:", x_train.shape)
print("Test Daten:", x_test.shape)
# Bei den Labeln ist es anders als bei den Bilddaten, hier haben wir nur einzelne Werte.
print("Trainings Label:", y_train.shape)
print("Test Label:", y_test.shape)

Trainings Daten: (60000, 28, 28)
Test Daten: (10000, 28, 28)
Trainings Label: (60000,)
Test Label: (10000,)


## Aufbau von Graustufenbildern

Der MNIST-Datensatz besteht aus handgeschriebenen Ziffern, die als Graustufenbildern dargestellt sind. In diesem Kontext bedeutet Schwarz, dass der Pixelwert den Wert 0 hat, und Weiß, dass der Pixelwert den Wert 255 hat. Grautöne sind zwischen diesen beiden Extremen und haben unterschiedliche Intensitäten.

Die Bilder im MNIST-Datensatz sind Matrizen, die aus 28x28 Pixeln bestehen.
Jeder dieser Pixel enthält einen einzigen Wert zwischen 0 und 255, welcher angibt, wie dunkel oder hell der Pixel ist. 

Schwarz-Weiß-Bilder sind im Umgang mit neuronalen Netzen von großer Bedeutung, da sie einfach als Matrizen dargestellt werden können, wodurch die Verarbeitung und Analyse dieser Bilder deutlich erleichtert wird.

Die Pixelwerte einer Bild-Matrix können verwendet werden, um Merkmale des Bildes zu extrahieren und um Muster zu erkennen.

In [ ]:
# Hier kannst du interaktiv ein Beispiel betrachten.
example = show_example(x_train, y_train, 0)

Struktur vom Bild: (28, 28)
Label: 5
Pixel in der Mitte: 240
Farbwerte (dunkel -> hell): 0 to 255


##Daten Normalisierung

Beim MNIST-Datensatz enthalten die Bilder die Graustufenwerte von 0 bis 255, wobei 0 Weiß und 255 Schwarz repräsentiert. Vor der Verwendung in einem neuronalen Netzwerk müssen die Bilder normalisiert werden, indem man die Graustufenwerte auf den Bereich von 0 bis 1 skaliert.

Warum wird das gemacht? Eine Normalisierung der Bilder hilft dem neuronalen Netzwerk, besser zu lernen, indem es vermeidet, dass bestimmte Werte bevorzugt werden, die aufgrund der Skalierung von größeren Werten in einigen Bildern verursacht werden könnten. Dies kann dazu beitragen, dass das Netzwerk insgesamt besser funktioniert und bessere Ergebnisse liefert.

In [ ]:
# Du kannst die Bildmatrizen normalisieren indem Du einfach durch den höchsten Farbwert teilst.
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# Hier siehst Du das normalisierte Bild
example = show_example(x_train, y_train, 0)

Struktur vom Bild: (28, 28)
Label: 5
Pixel in der Mitte: 0.9411764705882353
Farbwerte (dunkel -> hell): 0.0 to 1.0


# Aufbau eines neuronalen Netzes

Neuronale Netze basieren auf der Funktionsweise von biologischen Neuronen im Gehirn. In einem neuronalen Netzwerk gibt es Eingangsneuronen, welche Daten aufnehmen, verarbeiten und an Ausgangsneuronen weitergeben. Die Neuronen sind in Schichten angeordnet, und jedes Neuron in einer Schicht ist mit jedem Neuron in der nächsten Schicht verbunden.

---
## Die Schichten
Ein neuronales Netz besteht aus verschiedenen Schichten, die aufeinander aufbauen und unterschiedliche Funktionen erfüllen. 

Die erste Schicht des Netzes ist die Eingangsschicht, welche die Daten aufnimmt. Die letzte Schicht des Netzes ist die Ausgangsschicht, welche das Ergebnis des Netzes ausgibt. Dazwischen befinden sich versteckte Schichten, welche die Eingabe durch komplexe mathematische Operationen verarbeiten und die Daten so transformieren, dass das Netzwerk ein sinnvolles Ergebnis ausgibt.

Jede Schicht des Netzes besteht aus Neuronen, welche Informationen von den vorherigen Schichten verarbeiten und an die nächste Schicht weiterleiten. Die Neuronen sind wie Bausteine in einem Lego-Set, welche aufeinander gestapelt werden und verschiedene Funktionen erfüllen. Wie bei einem Lego-Set können die Schichten und Neuronen auch in neuronalen Netzen je nach Problemstellung und Datenstruktur unterschiedlich angeordnet werden, um die kreativsten Figuren zu erzeugen.


In [ ]:
# Hier kannst Du das finale Modell sehen.
model = models.Sequential([ # Sequentiell bedeutet, dass das Neuronal schichtweise aufgebaut wird.
    layers.Flatten(input_shape=(28, 28)), # Das Bild wird in die Länge gezogen -> 28*28 Pixel = 1*784.
    layers.Dense(128, activation='relu'), # Die erste Dichteschicht besteht aus 128 Neuronen.
    layers.Dense(10, activation='softmax') # Hier siehst Du die finale Ausgabeschicht.
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


# Interpretation
Der gezeigte Code erstellt ein neuronales Netz mit drei Layern: der Eingangsschicht, einer versteckten Schicht (Layer) und der Ausgabeschicht. Die Eingangsschicht ist eine Flattening-Schicht, die das ursprüngliche 28x28 Pixel große Bild in einen eindimensionalen Vektor mit 784 Werten umwandelt.

Die versteckte Schicht besteht aus 128 Neuronen und verwendet die ReLU-Aktivierungsfunktion.

Die Ausgabeschicht besteht aus 10 Neuronen, welche nummerische Aussagen des neuronalen Netzes bezüglich 10 möglichen Klassen ausgeben.

## Aktivierungsfunktionen

Wie Dir aufgefallen sein wird hat das neuronale Netz zwei unterschiedliche Aktivierungsfunktionen.

ReLU (Rectified Linear Unit) ist eine Aktivierungsfunktion, die oft in neuronalen Netzen verwendet wird, um die Ausgabe von Neuronen zu berechnen. Sie wird oft in tieferen Schichten von neuronalen Netzen verwendet, um die Ausgabe von Neuronen zu aktivieren und zu berechnen.

Die Softmax-Funktion ist eine Aktivierungsfunktion, die normalerweise am Ende des neuronalen Netzes verwendet wird, um eine Wahrscheinlichkeitsverteilung für die Ausgabe des Netzes zu erzeugen. Die Softmax-Funktion wird auf die Ausgabe von einem oder mehreren Neuronen angewendet und berechnet die Wahrscheinlichkeiten für jede mögliche Ausgabe. 

## Parameter des Netzes

Die endgültigen Parameter des Modells (Gewichte, Bias) werden während des Trainings auf den Trainingsdaten erlernt und hängen generell von dem Aufbau des neuronalen Netzes ab.

Die Anzahl der Parameter in den Schichten eines neuronalen Netzes ergibt sich dabei aus der Anzahl an Neuronen in jeder Schicht, der Anzahl der Eingabevariablen, der Art der Schicht und der Art der Aktivierungsfunktion.

> **_Achtung:_**  In einem neuronalen Netz geben die einzelnen Schichten ihre Ausgaben an die nächste Schicht weiter.

Für eine Schicht mit $n$ Neuronen und $m$ Eingabevariablen gibt es $n*m$ Gewichte, die trainiert werden müssen. Hinzu kommt noch, dass die $n$ Neuronen einer Schicht selber als Parameter hinzugezählt werden müssen. Somit ergibt sich die Gesamtzahl an Parametern zwischen zwei Sichten aus: $(n*m)+n$ wobei $n$ die Anzahl der Neuronen und $m$ die Anzahl der Eingabevariablen ist.

Beispiel:

Die erste Flatten-Schicht gibt 784 Pixel (flache 28x28 Bilder) an die erste Dense-Schicht aus 128 Neuronen weiter. Somit erzeugt diese Schicht $(128 * 784) + 128 = 100.480$ Parameter.

# Ausblick

Du hast in diesem Tutorial gesehen, wie ein neuronales Netz aufgebaut wird und mit Eingabe-Daten umgeht.

Sobald man das Verständnis dafür hat, wie das untrainierte Modell funktioniert, gibt es viele spannende Möglichkeiten, um es zu optimieren und seine Leistung zu verbessern.

Obwohl das Modell noch nicht gelernt hat, wofür es eigentlich eingesetzt wird, kann es Bilder als Eingabe entgegennehmen und in eine den 10 Zielklassen angemessene Darstellung bringen. Dies geschieht mittels [softmax](!https://developers.google.com/machine-learning/glossary?hl=de#softmax) welche dafür sorgt, dann die "rohen" Ausgaben des Netzes direkt in interpretirebare Wahrscheinlicheiten überführt werden.

Je höher ein Wert für eine bestimmte Ausgabe ist, desto stärker ist das Netzwerk davon überzeugt, dass diese Ausgabe korrekt ist. Umgekehrt gilt: Je niedriger dieser Wert ist, desto unwahrscheinlicher ist es, dass das Netzwerk diese Ausgabe als korrekt ansieht.

Wenn das neuronale Netzwerk trainiert wird, ist das Ziel, die Ausgabe-Werte so zu optimieren, dass sie so nah wie möglich an den tatsächlichen Werten der Trainingsdaten liegen. Auf diese Weise kann das Netzwerk lernen, Muster in den Daten zu erkennen und genaue Vorhersagen zu treffen.

Dazu aber mehr in der nächsten Woche.


In [ ]:
# So kann das Modell unser Beispiel verarbeiten.
predictions = model(example).numpy().round(2)
print("Logits:", predictions)
# Du kannst sehen, dass 10 Ausgaben direkt als Wahrscheinlichkeiten interpretierbar sind (Summe == 1).
print("Summe Logits:", predictions.sum().round())

Logits: [[0.13 0.07 0.1  0.06 0.13 0.13 0.07 0.12 0.11 0.08]]
Summe Logits: 1.0


In [ ]:
# Wie Du siehst, hat das neuronale Netz noch nichts gelernt.
print("Beispiel echtes Label:", y_train[0])
print("NN geschätztes Label:", np.argmax(predictions))

Beispiel echtes Label: 5
NN geschätztes Label: 0
